In [1]:
from haystack.utils import   print_answers
from haystack.nodes import FARMReader
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever, PreProcessor
from haystack.pipelines import ExtractiveQAPipeline
from pymongo import MongoClient
from haystack.schema import Document


c:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [2]:

db_client = MongoClient(host="localhost", port=27017)
database = db_client['Website_Chatbot']
collection = database["MitsSpider"]


In [3]:
docs = [] 
for document in collection.find({}):
    content = document["content"]
    for para in content.split("\n\n"):
        if not para.strip():
            continue

        tmp_doc = Document(content=para)
        tmp_doc.id = str(document["_id"])
        tmp_doc.meta = {"source":document['source'], }
        tmp_doc.content_type = "str"
        docs.append(tmp_doc)


In [16]:

new_docs = [] 
for document in collection.find({}):
    content = document["content"]
    for para in content.split("\n\n"):
        if not para.strip():
            continue

        tmp_doc = dict() #Document(content=para)
        tmp_doc["content"] = para
        tmp_doc["id"] = str(document["_id"])
        tmp_doc["meta"] = {"source":document['source'],"name":str(document["_id"]) }
        # tmp_doc.content_type = "str"
        new_docs.append(tmp_doc)

In [9]:
preprocessor = PreProcessor(clean_header_footer=True, split_by=None)
clean_docs = preprocessor.process(docs)
# preprocessor.clean(docs, clean_whitespace=True, clean_header_footer=True, clean_empty_lines=True)


TypeError: 'Document' object is not subscriptable

In [4]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
document_store.write_documents(docs)

Writing Documents: 10000it [00:00, 41031.36it/s]         


In [5]:
retriever = DensePassageRetriever(document_store=document_store,
                                 query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
                                 passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
                                 max_seq_len_query=64,
                                 max_seq_len_passage=512,
                                 batch_size=16,
                                 use_gpu=True,
                                 embed_title=True,
                                 use_fast_tokenizers=True,
                                 similarity_function="cosine")
document_store.update_embeddings(retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-ctx_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-ctx_encoder

In [6]:
reader = FARMReader(model_name_or_path="Saved Models/roberta_base_squad2", use_gpu=True,
num_processes=0)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at Saved Models/roberta_base_squad2
INFO - haystack.modeling.model.language_model -  Loaded Saved Models/roberta_base_squad2
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
INFO - haystack.modeling.model.prediction_head -  Loading prediction head

In [7]:
pipeline = ExtractiveQAPipeline(reader=reader,retriever=retriever)

In [10]:
prediction = pipeline.run(query="Who is the principal of MITS",
                         params = {"Retriever":{"top_k":10}, 
                                  "Reader":{"top_k":10}})

print_answers(prediction)                                

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.55 Batches/s]


Query: Who is the principal of MITS
Answers:
[   <Answer {'answer': 'Dr Chikku Abraham', 'type': 'extractive', 'score': 0.6559404730796814, 'context': 'llege Executive Director Mr.P George Varghese, the acting Pricipal Dr Chikku Abraham and the HODs of various departments were the dignitaries on the d', 'offsets_in_document': [{'start': 3880, 'end': 3897}], 'offsets_in_context': [{'start': 67, 'end': 84}], 'document_id': '6213cd02abf8a36f04f5c6d4', 'meta': {'source': 'http://mgmits.ac.in/life-mits/sports/', 'vector_id': '32'}}>,
    <Answer {'answer': 'Dr Chikku Abraham', 'type': 'extractive', 'score': 0.20968271791934967, 'context': 'Vice Principal Muthoot Institute of Technology and Science (MITS) Dr Chikku Abraham is currently the Vice- Principal and Associate Professor in Electr', 'offsets_in_document': [{'start': 66, 'end': 83}], 'offsets_in_context': [{'start': 66, 'end': 83}], 'document_id': '6213cd07abf8a36f04f5c750', 'meta': {'source': 'http://mgmits.ac.in/mits/executive-body

### Saving the models

In [26]:
retriever.save("context_model_retriever_2")
document_store.save("document_store")

INFO - haystack.modeling.model.biadaptive_model -  prediction_head saving


In [2]:
tmp_doc_store = FAISSDocumentStore.load("document_store")
tmp_reader = FARMReader(model_name_or_path="Saved Models/roberta_base_squad2",
use_gpu=True, num_processes=0)
tmp_retriever = DensePassageRetriever.load("context_model_retriever_2", tmp_doc_store)
tmp_pipeline = ExtractiveQAPipeline(tmp_reader, tmp_retriever)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at Saved Models/roberta_base_squad2
INFO - haystack.modeling.model.language_model -  Loaded Saved Models/roberta_base_squad2
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
INFO - haystack.modeling.model.prediction_head -  Loading prediction head

In [3]:
predictions = tmp_pipeline.run("Where is MITS located")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]C:\Users\ANT-PC\anaconda3\envs\ds\lib\site-packages\haystack\modeling\model\prediction_head.py:462: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.26s/ Batches]


In [7]:
answers = predictions["answers"]


In [9]:
answers[0].document_id

'6213cd05abf8a36f04f5c720'